In [12]:
import tensorflow as tf
import numpy as np

def ttq_reference(w, thres_ratio):
    G = tf.get_default_graph()
    shape = w.get_shape()

    thres = tf.reduce_max(tf.abs(w)) * thres_ratio

    w_p = tf.get_variable('w_p', initializer=1.0)
    w_n = tf.get_variable('w_n', initializer=1.0)
    
    ones = tf.constant(1.0, shape=shape)
    
    mask_p = tf.where(w > thres, ones * w_p, ones)
    mask_np = tf.where(w < -thres, ones * w_n, mask_p)
    mask_z = tf.cast((w > thres) | (w < -thres), tf.float32)
    w_ = tf.assign(mask_z, )

    with G.gradient_override_map({'Sign': 'Identity', 'Mul': 'Add'}):
        w_ter = tf.sign(w) * mask_z
    return tf.get_variable('w_ter', initializer=w_ter * mask_np, trainable=False)

In [13]:
tf.reset_default_graph()
with tf.Session() as sess:
    w = tf.get_variable('w', initializer=[1.0, 2.0, 3.0, -2.0])
    
    w_ter = ttq_reference(w, 0.5)

    sess.run(tf.global_variables_initializer())
    variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    
    print([v.name for v in variables])
    print(sess.run(tf.gradients(w_ter, variables, 1.0)))
    
    saver = tf.train.Saver({'w' : w_ter})

['w:0', 'w_p:0', 'w_n:0']


ValueError: Shapes () and (4,) are not compatible